<a href="https://colab.research.google.com/github/thinkGrow/Neural-Network-CSC424/blob/main/nn_2b_assignment_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
x_train = trainset.data
x_test = testset.data
y_train = np.array(trainset.targets)
y_test = np.array(testset.targets)
x_train = x_train/255
x_test = x_test/255

In [ ]:
x_train = x_train[np.any([y_train==3,y_train==5], axis=0)] #store all x_train with class 3 and 5
y_train = y_train[np.any([y_train==3,y_train==5], axis=0)] #store all y_train with class 3 and 5
x_test =  x_test[np.any([y_test==3,y_test==5], axis=0)]
y_test = y_test[np.any([y_test==3,y_test==5], axis=0)]

y_train = np.where(y_train==3, 0, 1) #class 3 becomes 0 and class 1 becomes 5.
y_test = np.where(y_test==3, 0, 1)

# class 0 = cat
# class 1 = dog

#checked and verified

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 3072) #10,000 x 3072
x_test = x_test.reshape(x_test.shape[0], 3072) #2000 x 3072
#print(x_test_rows.shape)
y_train = y_train.reshape(10000,1)
y_test = y_test.reshape(2000,1)

In [ ]:
def sigmoid( signal ):
    # Prevent overflow.
    signal = np.clip( signal, -500, 500 )
    # Calculate activation signal
    signal = 1.0/( 1 + np.exp( -signal ))
    return signal


def sigmoid_der(x):
    return x *(1-x)

In [ ]:
class NeuralNetwork:
    def __init__(self):
        #random.rand = random values given in a shape.

        fan_in_h = 3072
        fan_out_h = 4000
        limit_h = math.sqrt(6/(fan_in_h + fan_out_h))

        fan_in_o = 4000
        fan_out_o = 1
        limit_o = math.sqrt(6/(fan_in_o + fan_out_o))

        self.wh = np.random.uniform(-limit_h, limit_h, size=(fan_in_h, fan_out_h)) #w1 to w(3,072*4,000) : rand(input,hidden)
        self.wo = np.random.uniform(-limit_o, limit_o, size=(fan_in_o, fan_out_o))
        self.lr = 0.000001  
        self.batch_size = 1000


    def train(self, x_train, y_train):
        num_of_iteration = 0 
        mul_of_iteration = 1
        
        for epoch in range(10):
            # x_train.shape = 10,000*3,072
            i = 0
            j = self.batch_size

            for i in range(10000//self.batch_size):
                x_batch = x_train[i:j,:]
                y_batch = y_train[i:j,:]
                i += self.batch_size
                j += self.batch_size

                #feed forward
                num_of_iteration += 1
                zh = np.dot(x_batch, self.wh)
                ah = sigmoid(zh) #10,000*4,000
                zo = np.dot(ah, self.wo) #10,000*1
                ao = sigmoid(zo) #10,000*1
                
                error_out = ( 0.5 * (np.power((ao - y_batch), 2))) 
                #prints out error after every mul_of_iteration-th iteration
                

                #back propagate

                #local gradients
                dcost_dao = ao - y_batch #10,000*1
                dzo_dwo = ah
                dao_dzo = sigmoid_der(ao) 
                dzo_dah = self.wo
                dzh_dwh = x_batch
                dah_dzh = sigmoid_der(ah)

                #upstream gradients = downstream gradients * local gradient
                dcost_dzo = dcost_dao * dao_dzo
                dcost_dwo = np.dot(dzo_dwo.T, dcost_dzo)
                dcost_dah = np.dot(dcost_dzo , dzo_dah.T)
                dcost_dzh = dah_dzh * dcost_dah
                dcost_dwh = np.dot(dzh_dwh.T, dcost_dzh)

                # Update Weights ================
                self.wh -= self.lr * dcost_dwh
                self.wo -= self.lr * dcost_dwo
            print( f'Epoch no. {epoch} : {error_out.sum()}' )
            
            
            
    def predict(self, x_test):
        # feedforward
        zh = np.dot(x_test, self.wh)
        ah = sigmoid(zh)

        zo = np.dot(ah, self.wo)
        ao = sigmoid(zo)            
        return ao
        

nn = NeuralNetwork()
nn.train(x_train,y_train)

Epoch no. 0 : 1253.319967016273
Epoch no. 1 : 1251.9909640512897
Epoch no. 2 : 1250.9081535425476
Epoch no. 3 : 1249.8543310248847
Epoch no. 4 : 1248.8283142427688
Epoch no. 5 : 1247.829206563545
Epoch no. 6 : 1246.8561389224628
Epoch no. 7 : 1245.9082683753143
Epoch no. 8 : 1244.9847777406944
Epoch no. 9 : 1244.084875203409


In [ ]:
#test
y_predict = nn.predict(x_test)

#binary classification
y_predict = np.where(y_predict<0.5, 0, 1 )
'''what np.where does : 
    if score < 0.5 : label 0  
    else : label 1  ''';

In [ ]:
#Accuracy
import pandas as pd
from sklearn.metrics import accuracy_score

print(f' Test Accuracy : { accuracy_score(y_test, y_predict) }')

 Test Accuracy : 0.5445
